In [2]:
# RTX GPU 和 Mac M GPU需要安装不同的Torch版本，查阅PyTorch官网。
import torch

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [4]:
import importlib

try:
    importlib.import_module("transformers")
    print("transformers exists.")
except ImportError:
    ! pip install transformers
    print("transformers installed.")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.13.2 which is incompatible.



   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/10.0 MB 12.7 MB/s eta 0:00:01
   ----------------------------- ---------- 7.3/10.0 MB 23.8 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 25.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.4 MB 2.1 MB/s eta 0:00:01
   ----------------- ---------------------- 1.0/2.4 MB 2.2 MB/s eta 0:00:01
   ---------------------- ----------------- 1.3/2.4 MB 2.2 MB/s eta 0:00:01
   ----------------------------------- ---- 2.1/2.4 MB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 2.3 MB/s eta 0:00:00


In [5]:
from transformers import BertTokenizerFast, BertForQuestionAnswering

C:\Users\32982\miniconda3\envs\2-BERT\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")

In [7]:
model = BertForQuestionAnswering.from_pretrained(
    "./fengchao-bert-qa"
).to(device)

In [8]:
# 定义一个名为 predict 的函数，接收两个参数 doc 和 query
def predict(doc, query):
    # 输出“段落：”和  doc 的内容
    # print("段落：", doc)
    # 输出“提问：”和 query 的内容
    print("提问：", query)
    # 将 doc 和 query 传递给 tokenizer 函数，将返回结果赋值给 item
    item = tokenizer(
        [doc, query], max_length=512, return_tensors="pt", truncation=True, padding=True
    )
    # 关闭 torch 的梯度计算
    with torch.no_grad():
        # 将 input_ids 和 attention_mask 传递给 model，将返回结果赋值给 outputs
        input_ids = item["input_ids"].to(device).reshape(1, -1)
        attention_mask = item["attention_mask"].to(device).reshape(1, -1)

        outputs = model(input_ids[:, :512], attention_mask[:, :512])

        ### START CODE HERE ###
        # 使用`torch.argmax`和它的`dim`参数来提取开始位置和结束位置的预测结果
        start_pred = torch.argmax(outputs.start_logits, dim=1)
        end_pred = torch.argmax(outputs.end_logits, dim=1)
        ### END CODE HERE ###

    # 将预测结果转为字符级别
    try:
        start_pred = item.token_to_chars(0, start_pred)
        end_pred = item.token_to_chars(0, end_pred)
    except:
        # 如果出现异常，则返回“无法找到答案”
        return "无法找到答案"

    # 判断结果是否有效，如果有效则返回结果
    if start_pred.start > end_pred.end:
        # 如果预测的开始位置大于结束位置，则返回“无法找到答案”
        return "无法找到答案"
    else:
        # 如果预测的开始位置小于结束位置，则返回预测的答案
        return doc[start_pred.start : end_pred.end]


In [9]:
doc = "芙宁娜是米哈游开发的电子游戏《原神》中的虚构角色。她是游戏中虚构国家枫丹的前最高统治者，是水神芙卡洛斯的人类分身与扮演者。退位后，芙宁娜不再扮演水神，过上普通人的生活。芙宁娜是游戏中兼具“荒性”和“芒性”两种水元素能力的角色，还可以在水面上移动。评论员对芙宁娜角色塑造的评价在早期偏向负面，浮夸和违和的表演型性格被批评缺乏神明的威严；在主线剧情揭晓芙宁娜身世后，评价转向正面，评论员称赞芙宁娜体现了女性坚忍不拔的精神力量。角色在战斗中强大的辅助能力得到评论员的正面评价。"

In [10]:
predict(doc= doc, query="水神的名字是什么")

提问： 水神的名字是什么


'芙卡洛斯'

In [11]:
predict(doc= doc, query="芙宁娜有几种元素能力")

提问： 芙宁娜有几种元素能力


'兼具“荒性”和“芒性”两种'

In [12]:
predict(doc= doc, query="为什么芙宁娜的评价转向正面")

提问： 为什么芙宁娜的评价转向正面


'在主线剧情揭晓芙宁娜身世后，评价转向正面，评论员称赞芙宁娜体现了女性坚忍不拔的精神力量。'

In [13]:
doc = "近年来，随着我国经济的快速发展，新能源汽车产业得到了前所未有的关注。政府出台了一系列扶持政策，旨在推动新能源汽车的研发和普及。截至2024年，我国新能源汽车产销量连续五年位居全球第一。新能源汽车不仅有助于缓解能源危机，还能有效降低环境污染。\n在新能源汽车领域，电动汽车占据了主导地位。目前，我国已形成了较为完善的电动汽车产业链，包括电池、电机、电控等关键零部件的生产。其中，动力电池作为电动汽车的核心部件，其性能直接影响着电动汽车的续航里程和安全性。我国动力电池企业在技术创新方面取得了显著成果，部分企业已具备与国际巨头竞争的实力。\n此外，充电基础设施建设也是新能源汽车产业发展的重要环节。近年来，我国充电桩数量迅速增加，覆盖了城市、乡村、高速公路等各个场景。然而，充电桩的布局和运营仍存在一些问题，如充电桩利用率低、充电接口不统一等。\n为进一步推动新能源汽车产业发展，政府提出了“新能源汽车下乡”活动，鼓励农村地区购买新能源汽车，以促进农村消费升级和绿色出行。同时，我国还在积极开展新能源汽车的国际合作，与国际先进企业共同研发新技术，提升我国新能源汽车产业的国际竞争力。"

In [14]:
predict(doc= doc, query="我国新能源汽车产销量连续几年位居全球第一")

提问： 我国新能源汽车产销量连续几年位居全球第一


'五年'

In [15]:
predict(doc= doc, query="新能源汽车产业发展的主要目的是什么？")

提问： 新能源汽车产业发展的主要目的是什么？


'推动新能源汽车的研发和普及'

In [16]:
predict(doc= doc, query="电动汽车在新能源汽车领域占据什么地位？")

提问： 电动汽车在新能源汽车领域占据什么地位？


'主导地位'

In [17]:
predict(doc= doc, query="我国动力电池企业在技术创新方面取得了哪些成果")

提问： 我国动力电池企业在技术创新方面取得了哪些成果


'我国动力电池企业在技术创新方面取得了显著成果，部分企业已具备与国际巨头竞争的实力。'

In [18]:
predict(doc= doc, query="充电基础设施建设在新能源汽车产业发展中扮演什么角色")

提问： 充电基础设施建设在新能源汽车产业发展中扮演什么角色


'充电基础设施建设也是新能源汽车产业发展的重要环节'

In [19]:
predict(doc= doc, query="政府提出“新能源汽车下乡”活动的作用？")

提问： 政府提出“新能源汽车下乡”活动的作用？


'鼓励农村地区购买新能源汽车，以促进农村消费升级和绿色出行。'